In [1]:
import json
import geopandas as gpd
import pandas as pd

# shapefile z czujnikami(sensors)
sensors = gpd.read_file("C:\\Users\\User\\Desktop\\studia_mix\\sem_7\\bazy_danych_przestrzennych\\04_azure_iot\\04_iot_data\\iot_sensors\\iot_sensors\\iot_sensors.shp")


sensors["sensor_id"] = pd.to_numeric(sensors["id"], errors="coerce").astype("Int64")

sensors = sensors.dropna(subset=["sensor_id"])

#konwersja na int
sensors["sensor_id"] = sensors["sensor_id"].astype(int)

print(sensors.head())



    id  type                   geometry  sensor_id
0  1.0  None  POINT (18.95809 50.27747)          1
1  2.0  None  POINT (18.95605 50.27781)          2
2  3.0  None  POINT (18.95672 50.28115)          3
3  4.0  None   POINT (18.9554 50.28192)          4
4  5.0  None   POINT (18.9555 50.27869)          5


In [3]:
# spr czy współrzędne są w WGS84 (lon/lat) ---
if sensors.crs is None:
    sensors = sensors.set_crs(4326)
else:
    sensors = sensors.to_crs(4326)

# precompute lon/lat z geometrii
sensors["lon"] = sensors.geometry.x
sensors["lat"] = sensors.geometry.y
sensors = sensors[["sensor_id", "lon", "lat"]].copy()


READINGS_JSON = r"C:\Users\User\Desktop\studia_mix\sem_7\bazy_danych_przestrzennych\04_azure_iot\04_iot_data\device_readings.json"
with open(READINGS_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)

preds = []
EPS = 1e-6       
MAX_DIST = None

for entry in data:
    df = pd.DataFrame(entry["readings"])
    if df.empty:
        preds.append({"timestamp": entry["timestamp"], "position": None})
        continue

    df["sensor_id"] = df["sensor_id"].astype(int)

    df = df.merge(sensors, on="sensor_id", how="inner")

    if df.empty:
        preds.append({"timestamp": entry["timestamp"], "position": None})
        continue

    # wagi odwrotne do odległości
    w = 1.0 / (df["distance_m"].values + EPS)
    w_sum = w.sum()

    # średnia ważona współrzędnych (x=lon, y=lat)
    lon_pred = float((df["lon"].values * w).sum() / w_sum)
    lat_pred = float((df["lat"].values * w).sum() / w_sum)

    preds.append({
        "timestamp": entry["timestamp"],
        "position": {"lat": lat_pred, "lon": lon_pred}
    })


OUT_JSON = r"C:\Users\User\Desktop\studia_mix\sem_7\bazy_danych_przestrzennych\04_azure_iot\04_iot_data\pred.json"
with open(OUT_JSON, "w", encoding="utf-8") as f:
    json.dump(preds, f, indent=2, ensure_ascii=False)

print(f"OK — zapisano {len(preds)} predykcji do: {OUT_JSON}")


OK — zapisano 243 predykcji do: C:\Users\User\Desktop\studia_mix\sem_7\bazy_danych_przestrzennych\04_azure_iot\04_iot_data\pred.json


In [13]:
import json
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# dane połączenia z Azure 
#usuniete, bo juz polaczone 

engine = create_engine(f"postgresql+psycopg2://{USER}:{PWD}@{HOST}:{PORT}/{DB}")


PATH = r"C:\\Users\\User\\Desktop\\studia_mix\\sem_7\\bazy_danych_przestrzennych\\04_azure_iot\\04_iot_data\\pred.json"
with open(PATH, "r", encoding="utf-8") as f:
    data = json.load(f)


for p in data:
    if p["position"] is not None:
        rows.append({
            "ts": p["timestamp"],
            "lat": p["position"]["lat"],
            "lon": p["position"]["lon"]
        })

df = pd.DataFrame(rows)


df.to_sql("predictions", engine, if_exists="append", index=False)
print(f"✅ Zapisano {len(df)} rekordów do bazy {DB}.")


✅ Zapisano 486 rekordów do bazy postgres.
